In [5]:
#Start by connecting gdrive into the google colab

from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
import pandas as pd
data = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Postdam.csv', header = None)  

import numpy as np

In [21]:
data = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Predam.csv', header = None) 

In [22]:
data = data.to_numpy()

In [17]:
import math
def dist(pref, x):
  if pref == 1:
    distance = (-0.8*(0.8-x[1])-(0.2-x[0])*0.2)/math.sqrt(0.64+0.04)
  elif pref == 2:
    distance = (-(1-x[1])-(1-x[0])*-1)/math.sqrt(2)
  elif pref == 3:
    distance = (0.2*(0.2-x[1])-(0.8-x[0])*0.8)/math.sqrt(0.04+0.64)
  
  distancemin = min(distance[distance>=0])
  distancemin2 = max(distance[distance<0])
  point1 = x[:,distance == distancemin]
  point2 = x[:,distance == distancemin2]
  return point1, point2

def intersect(pref, a1, b1, c1):
   
  if pref == 1:
    a2 = 0.2
    b2 = -0.8
    c2 = 0.2*0.2 - 0.8*0.8
  elif pref == 2:
    a2 = 1
    b2 = -1
    c2 = 0
  elif pref == 3:
    a2 = 0.8
    b2 = -0.2
    c2 = 0.8*0.8 - 0.2*0.2
  delta = a1*b2-a2*b1
  inter_x = (b2*c1 - b1*c2)/delta
  inter_y = (a1*c2 - a2*c1)/delta
  return inter_x, inter_y 

In [29]:
rec = np.zeros((1,6))
for i in range(48):
  inflownumber = i+1
  datayear = -data[data[:,19]==inflownumber]
  if len(datayear) == 0:
    continue
  energymax = max(datayear[:,17])
  energymin = min(datayear[:,17])
  if energymin == energymax:
    continue
  energyutility = (datayear[:,17]-energymin)/(energymax-energymin)
  navimax = max(datayear[:,18])
  navimin = min(datayear[:,18])
  naviutility = (datayear[:,18] - navimin)/(navimax-navimin)
  utility = np.vstack([energyutility,naviutility])
  #print(utility, inflownumber)
  for prefer in range(5):
    pref = prefer + 1
    if pref < 4:
      point1, point2 = dist(pref, utility)
      a1 = point2[1] - point1[1]
      b1 = point1[0] - point2[0]
      c1 = a1*point1[0] + b1*point1[1]
      [x_inters], [y_inters] = intersect(pref, a1, b1, c1)
    elif pref == 4:
      x_inters = 0.0
      y_inters = 1.0
    elif pref == 5:
      x_inters = 1.0
      y_inters = 0.0 
    energy_profit = x_inters *(energymax-energymin) + energymin
    navi_profit = y_inters *(navimax - navimin) + navimin
    print(x_inters,y_inters,energy_profit,navi_profit, inflownumber, pref)
    rec = np.vstack([rec,[x_inters,y_inters,energy_profit,navi_profit, inflownumber, pref]])
    

  



0.31865602315007735 0.8296640057875195 13.037051987869434 2.1105185408955776 1 1
0.6516589470494634 0.6516589470494634 13.037189980951068 2.0682102985347166 1 2
0.8821171631195726 0.5284686524782897 13.037285480531226 2.0389304293210397 1 3
0.0 1.0 13.03691994 2.151004 1 4
1.0 0.0 13.03733433 1.9133240000000002 1 5
0.19999999999999996 0.8 12.597891488 2.127236 2 1
0.5 0.5 12.598084745000001 2.091584 2 2
0.8 0.19999999999999996 12.598278002 2.055932 2 3
0.0 1.0 12.59776265 2.151004 2 4
1.0 0.0 12.59840684 2.032164 2 5
0.26395483125757646 0.8159887078143944 12.553638691347363 2.107268196073325 4 1
0.5892294471482622 0.5892294471482622 12.553694664603267 2.053372054998199 4 2
0.8266353142359895 0.30654125694395756 12.553735517404874 1.9861827259504399 4 3
0.0 1.0 12.55359327 2.151004 4 4
1.0 0.0 12.55376535 1.9133240000000002 4 5
0.2826967275923382 0.8206741818980846 13.33374530191529 2.1083818395535365 5 1
0.611867819496875 0.611867819496875 13.333777662725334 2.058752743338017 5 2
0.842

In [30]:
from numpy import savetxt
savetxt('predata.csv', rec, delimiter=',')